# Tutorial: Valve Unit Model with IAPWS Property Package

![](valve_1.svg)

## Learning Outcomes

- Demonstrate use of the valve unit model in IDAES


## Problem Statement

In this example, we will be passing a liquid stream through a valve.

**Stream Inputs:**

Flow Rate = 1000 mol/s

Temperature = 298 K

Inlet Pressure = 202650 Pa

Outlet Pressure = 101325 Pa

Valve Opening = 0.5 (fraction)

For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Importing necessary tools

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import math functions
import math

# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, SolverFactory, units

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the Valve unit model
from idaes.models.unit_models import Valve

# Import the option to set the valve function
from idaes.models.unit_models import ValveFunctionType

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the iapws95 property package to create a property block for the flowsheet
from idaes.models.properties import iapws95

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

## Setting up the flowsheet
In the following cell, we will create the `ConcreteModel` foundation, attach the steady state flowsheet, and declare the property parameter block that will used.

More information on this general workflow can be found here: https://idaes-pse.readthedocs.io/en/stable/how_to_guides/workflow/general.html

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

m.fs.properties = iapws95.Iapws95ParameterBlock()

In the following cell, we will be creating the valve unit model, assigning the valve function and property package to it, and determining the initial degrees of freedom associated with the valve unit model. The `ValveFunctionType` determines how the flow coefficient will be calculated.

In [ ]:
m.fs.valve = Valve(
    valve_function_callback=ValveFunctionType.linear,
    property_package=m.fs.properties
)

DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom is: {0}'.format(DOF_initial))

In [ ]:
assert DOF_initial == 3

### Fixing input specifications
In the following cell, we will be calculating and specifying the inlet conditions for the valve block and re-evaluating the degrees of freedom to ensure the problem is square (i.e. DOF=0).

In [ ]:
# Assign the known inlet conditions to variables
fin = 1000  # mol/s
pin = 202650  # Pa
pout = 101325  # Pa
tin = 298  # K

# Determine inlet enthalpy
hin = iapws95.htpx(T=tin * units.K, P=pin * units.Pa)  # J/mol

# Determine flow coefficient - equation depends on the valve function specified earlier
cv = 1000 / math.sqrt(pin - pout) / 0.5

# Fix the inlet conditions
m.fs.valve.inlet.enth_mol[0].fix(hin)
m.fs.valve.inlet.flow_mol[0].fix(fin)
m.fs.valve.inlet.pressure[0].fix(pin)
m.fs.valve.outlet.pressure[0].set_value(pout) # Sets the target value for the outlet pressure, but does not fix it
m.fs.valve.Cv.fix(cv)
m.fs.valve.valve_opening.fix(0.5)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization
IDAES includes pre-written initialization routines for all unit models. Failing to initialize or having a poor intialization of a flowsheet may result in the problem being unsolvable. The output from initialization can be set to 7 different levels depending on the details required by the user. In general, when a particular output level is set, any information at that level and above gets picked up by logger. The default level taken by the logger is INFO. 

More information on these levels can be found in the IDAES documentation: 
https://idaes-pse.readthedocs.io/en/stable/reference_guides/logging.html?highlight=log%20level#idaes-solve-loggers

In [ ]:
m.fs.valve.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results
In the following cell, the flowsheet will be solved using the IDAES `get_solver` tool. Setting `tee=True` will display the solver output.

In [ ]:
solver = get_solver()
result = solver.solve(m, tee=True)

In [ ]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

As expected, the report will show that pressure of the stream is halved after going through the valve.

In [ ]:
m.fs.valve.report()

In [ ]:
import pytest

# Check results
assert value(m.fs.valve.outlet.flow_mol[0]) == pytest.approx(1000, rel=1e-6)
assert value(m.fs.valve.outlet.pressure[0]) == pytest.approx(101325, rel=1e-6)
assert value(m.fs.valve.outlet.enth_mol[0]) == pytest.approx(1880.557, rel=1e-3)